In [69]:
import torch
import torch.nn as nn
import pickle
import numpy as np
import os
import re

import transformers
from transformers import AutoTokenizer, AutoModel

In [40]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [15]:
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')

In [17]:
example = pickle.load(open('./data/pretrained_ehr/mimic_12_lab_150_2020.pkl', 'rb'))

In [60]:
eicu = pickle.load(open('./data/pretrained_ehr/eicu_12_lab_150_2020.pkl', 'rb'))
eicu.head()

,ID,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,...,los>7day_fold,dx_fold,code_name,value,value_uom,order_offset,lab_name_12hr,order_offset_12hr,lab_offset_order_12hr,lab_id_12hr
0,1590305,1230929,Male,48.0,Caucasian,252,628,"Sepsis, pulmonary",177.8,17:06:00,...,1,1,"[CPK-MB INDEX, CPK-MB, CPK, troponin - I, paCO...","[0.4, 5.7, 1546, <0.012, 68.8, 31, 100, 1, 68,...","[%, ng/mL, Units/L, ng/mL, mm Hg, null, %, mEq...","[53, 53, 53, 53, 61, 61, 61, 61, 61, 61, 61, 6...","[CPK-MB INDEX, CPK-MB, CPK, troponin - I, paCO...","[53, 53, 53, 53, 61, 61, 61, 61, 61, 61, 61, 6...","[1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, ...","[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,..."
1,3192089,2596335,Female,60.0,Caucasian,443,1063,Seizures (primary-no structural brain disease),168.0,19:40:58,...,1,1,"[sodium, FiO2, Base Excess, HCO3, Methemoglobi...","[130, 60, -2.7, 22.6, 1.6, 40.0, 193, 3.2, 7.3...","[mmol/L, %, mEq/L, mmol/L, %, mm Hg, mm Hg, %,...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 289...","[sodium, FiO2, Base Excess, HCO3, Methemoglobi...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 289...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, ...","[39, 10, 11, 44, 45, 8, 12, 46, 13, 47, 42, 21..."
2,1109620,824527,Male,61.0,Caucasian,199,487,"Bleeding, lower GI",188.0,06:28:00,...,1,1,"[MCH, RBC, Hct, RDW, MCHC, WBC x 1000, MPV, tr...","[30.3, 3.23, 28.9, 13.9, 33.9, 11.98, 10.2, <0...","[pg, M/mcL, %, %, g/dL, K/mcL, fL, ng/mL, fL, ...","[105, 105, 105, 105, 105, 105, 105, 105, 105, ...","[MCH, RBC, Hct, RDW, MCHC, WBC x 1000, MPV, tr...","[105, 105, 105, 105, 105, 105, 105, 105, 105, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, ...","[40, 16, 33, 30, 38, 35, 29, 50, 37, 5, 32, 32..."
3,1033801,765057,Female,52.0,Caucasian,188,434,"Sepsis, other",160.0,15:08:00,...,1,1,"[lactate, WBC's in urine, urinary specific gra...","[3.0, >100, 1.008, 102, 126, -13.0, 31.4, 98, ...","[mmol/L, null, null, mg/dL, mm Hg, mEq/L, mm H...","[1, 16, 16, 76, 77, 77, 77, 77, 77, 77, 77, 77...","[lactate, WBC's in urine, urinary specific gra...","[1, 16, 16, 76, 77, 77, 77, 77, 77, 77, 77, 77...","[1, 2, 2, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5]","[34, 55, 56, 42, 12, 11, 8, 14, 13, 10, 44, 9,..."
4,1788369,1404609,Female,87.0,Caucasian,283,678,"Sepsis, pulmonary",154.9,15:41:00,...,1,1,"[BUN, BNP, anion gap, Total CO2, potassium, cr...","[19, 1962 , 10, 28, 5.0, 0.92, 106, 9.5, 3.0,...","[mg/dL, pg/mL, null, null, mmol/L, mg/dL, mmol...","[135, 135, 135, 135, 135, 135, 135, 135, 135, ...","[BUN, BNP, anion gap, Total CO2, potassium, cr...","[135, 135, 135, 135, 135, 135, 135, 135, 135, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3]","[31, 49, 22, 9, 21, 17, 36, 19, 26, 54, 39, 24..."


In [19]:
example_list = example['code_name'].values.tolist()
ex = example_list[0]

In [61]:
eicu_list = eicu['code_name'].values.tolist()
eicu_ex = eicu_list[0]

In [5]:
mimic_dict = pickle.load(open('./data/pretrained_ehr/mimic_lab_word2embed.pkl', 'rb'))
eicu_dict = pickle.load(open('./data/pretrained_ehr/eicu_lab_word2embed.pkl', 'rb'))

In [54]:
print('mimic dictionary length: ', len(mimic_dict))
print('eicu dictionary length: ', len(eicu_dict))

mimic dictionary length:  545
eicu dictionary length:  157


In [122]:
mimic_id_dict = {}
mimic_embed_weight = torch.empty(len(mimic_dict), 768)
mimic_key_list = list(mimic_dict.keys())

num = 0
for i in range(len(mimic_dict)):
    name = mimic_key_list[i]
    cls_embed = mimic_dict[name]
    
    mimic_id_dict[name] = num
    num += 1
    
    mimic_embed_weight[i] = torch.Tensor(cls_embed)

In [123]:
eicu_id_dict = {}
eicu_embed_weight = torch.empty(len(eicu_dict), 768)
eicu_key_list = list(eicu_dict.keys())

num = 0
for i in range(len(eicu_dict)):
    name = eicu_key_list[i]
    cls_embed = eicu_dict[name]
    
    eicu_id_dict[name] = num
    num += 1
    
    eicu_embed_weight[i] = torch.Tensor(cls_embed)

In [111]:
lab_id_dict = {}
embed_weight = torch.empty(678, 768)
key_list = list(mimic_dict.keys()) + list(eicu_dict.keys())

num = 0
for i in range(len(key_list)):
    name = key_list[i]
    
    if name not in lab_id_dict:
        if i < 545:
            cls_embed = mimic_dict[name]
            
        else:
            cls_embed = eicu_dict[name]
        
        lab_id_dict[name] = num
        embed_weight[num] = torch.Tensor(cls_embed)
        num += 1

In [120]:
key_list = list(mimic_dict.keys()) + list(eicu_dict.keys())
len(key_list)

702

In [84]:
list(mimic_dict.keys())[-1]

'SURFACTANT ALBUMIN RATIO'

In [104]:
lab_id_dict['Base Excess']

66

In [102]:
mimic_dict['Base Excess']

[-0.08252817,
 -0.14111973,
 0.7499439,
 -0.9998615,
 0.9981124,
 0.9829474,
 0.055783268,
 -0.8827632,
 0.14469397,
 -0.061499454,
 0.9869634,
 0.99841934,
 -0.99823415,
 -0.37429628,
 -0.24529026,
 -0.096755184,
 0.9998746,
 0.0527951,
 -0.88261944,
 0.25191948,
 -0.08527811,
 -0.5286507,
 0.044399533,
 0.99986506,
 0.20379417,
 -0.062139824,
 0.9991706,
 0.86652374,
 -0.25860482,
 -0.0098816445,
 -0.13061312,
 -0.99521476,
 0.9995359,
 -0.9936817,
 -0.12586889,
 -0.032880038,
 -0.07305014,
 -0.2349978,
 0.92102456,
 -0.9997891,
 0.108198546,
 -0.5737085,
 0.021777669,
 -0.05178956,
 0.9998129,
 -0.12531854,
 -0.038082343,
 -0.05867324,
 -0.040248003,
 0.1728277,
 0.14989945,
 0.08994766,
 -0.9999021,
 0.92554677,
 0.9996436,
 -0.077581584,
 0.99940413,
 -0.10165929,
 -0.9752471,
 -0.038788173,
 -0.009441937,
 -0.044672165,
 -0.3410935,
 0.015240486,
 -0.05839327,
 0.21978135,
 -0.9981584,
 0.17083572,
 0.15752496,
 -0.052150983,
 0.037884872,
 0.105321474,
 0.5370081,
 0.2788875,
 -

In [100]:
eicu_dict['Base Excess']

[-0.08252808,
 -0.14111967,
 0.7499437,
 -0.9998615,
 0.9981124,
 0.9829474,
 0.05578334,
 -0.88276315,
 0.14469415,
 -0.061499465,
 0.98696333,
 0.99841934,
 -0.99823415,
 -0.3742959,
 -0.24529019,
 -0.096755125,
 0.9998746,
 0.052795164,
 -0.8826193,
 0.25191945,
 -0.08527812,
 -0.52865046,
 0.044399463,
 0.99986506,
 0.20379399,
 -0.062139727,
 0.9991706,
 0.8665235,
 -0.2586048,
 -0.009881872,
 -0.13061309,
 -0.99521476,
 0.9995359,
 -0.9936817,
 -0.12586898,
 -0.032880027,
 -0.07305015,
 -0.23499762,
 0.9210245,
 -0.9997891,
 0.10819883,
 -0.5737088,
 0.021777803,
 -0.05178956,
 0.9998129,
 -0.12531823,
 -0.038082253,
 -0.058672998,
 -0.040248062,
 0.17282736,
 0.1498994,
 0.08994645,
 -0.9999021,
 0.92554665,
 0.9996436,
 -0.07758165,
 0.99940413,
 -0.10165935,
 -0.9752471,
 -0.038788106,
 -0.009441971,
 -0.04467212,
 -0.34109363,
 0.015240652,
 -0.05839305,
 0.21978118,
 -0.9981584,
 0.17083538,
 0.15752491,
 -0.052150887,
 0.03788501,
 0.10532121,
 0.5370077,
 0.27888745,
 -0.0

In [86]:
exx = re.sub(r'[.,/|!-?"\':;~()\[\]]', '', eicu_ex[1])
idd = lab_id_dict[exx]
print('id: ', idd)
embed_weight[idd]

id:  584


tensor([ 1.5195e-01, -7.5401e-02,  9.9622e-01, -9.9999e-01,  9.9985e-01,
         8.8763e-01, -1.1112e-01,  3.9848e-01, -3.7925e-01, -7.9745e-02,
         9.9983e-01,  9.9998e-01, -9.6280e-01, -9.7739e-01, -1.9013e-01,
        -8.6979e-01,  1.0000e+00, -2.0767e-03, -9.9938e-01, -6.3968e-02,
         1.2108e-02, -9.8823e-01,  1.3364e-01,  9.9950e-01,  3.4575e-02,
        -1.7734e-01,  9.9999e-01,  9.9950e-01, -6.8827e-02, -1.6255e-01,
        -1.6341e-01, -9.9986e-01,  9.9837e-01, -9.9992e-01, -2.8505e-02,
         1.6069e-01, -3.7263e-02, -5.1431e-02,  7.7016e-01, -9.9877e-01,
        -2.4158e-02,  2.3349e-01,  6.2571e-02,  6.1475e-02,  9.9978e-01,
        -1.3984e-01,  1.8712e-03, -8.0096e-03, -7.0188e-02,  9.5247e-01,
         1.5003e-01,  9.9967e-01, -9.9972e-01,  9.9989e-01,  1.0000e+00,
         1.0592e-01,  9.9999e-01, -9.1564e-02, -3.9887e-01, -1.6435e-01,
         6.2495e-02,  7.9741e-02, -3.9250e-01,  9.9116e-02,  8.8606e-02,
         2.1378e-02, -9.8876e-01,  1.7795e-01, -9.7

In [87]:
## check the real CLS output from ClinicalBERT

ex1 = tokenizer([exx], padding=True, return_tensors='pt')
_, CLS = model(**ex1)

In [89]:
CLS[0]

tensor([ 1.5195e-01, -7.5401e-02,  9.9622e-01, -9.9999e-01,  9.9985e-01,
         8.8763e-01, -1.1112e-01,  3.9848e-01, -3.7925e-01, -7.9745e-02,
         9.9983e-01,  9.9998e-01, -9.6280e-01, -9.7739e-01, -1.9013e-01,
        -8.6979e-01,  1.0000e+00, -2.0767e-03, -9.9938e-01, -6.3968e-02,
         1.2108e-02, -9.8823e-01,  1.3364e-01,  9.9950e-01,  3.4574e-02,
        -1.7734e-01,  9.9999e-01,  9.9950e-01, -6.8827e-02, -1.6255e-01,
        -1.6341e-01, -9.9986e-01,  9.9837e-01, -9.9992e-01, -2.8505e-02,
         1.6069e-01, -3.7263e-02, -5.1431e-02,  7.7016e-01, -9.9877e-01,
        -2.4158e-02,  2.3349e-01,  6.2571e-02,  6.1475e-02,  9.9978e-01,
        -1.3984e-01,  1.8714e-03, -8.0095e-03, -7.0188e-02,  9.5247e-01,
         1.5003e-01,  9.9967e-01, -9.9972e-01,  9.9989e-01,  1.0000e+00,
         1.0592e-01,  9.9999e-01, -9.1564e-02, -3.9887e-01, -1.6435e-01,
         6.2495e-02,  7.9741e-02, -3.9250e-01,  9.9116e-02,  8.8605e-02,
         2.1379e-02, -9.8876e-01,  1.7795e-01, -9.7

In [124]:
# save tensor 
# mimic
pickle.dump(mimic_id_dict, open('./data/pretrained_ehr/mimic_lab_id_dict.pkl', 'wb'))
pickle.dump(mimic_embed_weight, open('./data/pretrained_ehr/mimic_lab_cls_initialized.pkl', 'wb'))

# eicu
pickle.dump(eicu_id_dict, open('./data/pretrained_ehr/eicu_lab_id_dict.pkl', 'wb'))
pickle.dump(eicu_embed_weight, open('./data/pretrained_ehr/eicu_lab_cls_initialized.pkl', 'wb'))

In [113]:
embed_weight.shape

torch.Size([678, 768])

In [112]:
len(lab_id_dict)

678

In [128]:
for name, age in mimic_id_dict.items():  # for name, age in dictionary.iteritems():  (for Python 2.x)
    if age == 68:
        print(name)

Calculated Total CO


In [119]:
embed_weight[406]

tensor([ 7.4477e-02, -3.6863e-02,  5.1146e-01, -9.9961e-01,  9.9711e-01,
         9.8925e-01,  9.5237e-02, -9.6020e-01, -1.7283e-02,  5.9738e-03,
         9.8511e-01,  9.9730e-01, -9.9929e-01, -3.7469e-03, -6.6110e-02,
        -1.0008e-01,  9.9974e-01,  1.5181e-02, -6.7952e-01,  1.7849e-01,
         3.2951e-02, -2.8700e-01,  1.2677e-01,  9.9991e-01, -5.7149e-02,
        -1.2460e-01,  9.9863e-01,  7.2641e-01, -3.3732e-01,  3.1909e-01,
        -1.6260e-01, -9.9337e-01,  9.9926e-01, -9.8743e-01, -1.9050e-02,
        -5.3732e-02, -9.2541e-03, -9.6559e-02,  8.8595e-01, -9.9963e-01,
         1.4810e-01, -7.2122e-01,  5.1684e-03, -7.2903e-02,  9.9989e-01,
        -9.8256e-03,  7.6757e-02, -5.2683e-02, -3.8095e-02, -1.9175e-01,
         9.7816e-02, -5.4109e-01, -9.9985e-01,  8.0454e-01,  9.9934e-01,
         5.0504e-02,  9.9937e-01, -4.1693e-03, -9.9423e-01, -2.0920e-02,
         1.4658e-01,  6.0501e-03, -2.5053e-01, -1.0250e-02, -1.3213e-01,
         1.5461e-01, -9.9447e-01,  7.2868e-02, -3.4

In [130]:
embed_weight[68]

tensor([-4.0764e-02, -7.4537e-02, -9.9783e-01, -9.9575e-01,  9.6933e-01,
         9.9998e-01,  3.5365e-02, -9.9999e-01, -6.2284e-02,  2.7783e-02,
         1.5780e-01, -3.2055e-01, -1.0000e+00,  9.9729e-01, -2.6222e-01,
         9.4585e-01,  9.9518e-01,  9.7876e-02,  9.9874e-01, -1.0648e-01,
        -3.7277e-02,  9.9908e-01,  4.6976e-02,  1.0000e+00,  3.9823e-02,
         1.3674e-01,  9.7183e-01, -9.9905e-01, -9.2087e-02, -1.6957e-01,
        -1.8033e-01, -7.6074e-01,  9.9996e-01,  5.6337e-01, -1.5751e-01,
         1.6004e-01,  9.6370e-02, -2.3354e-02,  9.9229e-01, -1.0000e+00,
         6.0977e-02, -9.9503e-01,  1.6913e-01,  1.3561e-01,  1.0000e+00,
        -7.4779e-02,  1.4936e-01,  1.8822e-01, -1.4571e-02, -9.9992e-01,
        -1.1881e-02, -1.0000e+00, -1.0000e+00, -9.9787e-01,  9.3991e-01,
         3.2722e-02,  9.8971e-01, -1.1052e-01, -1.0000e+00,  1.0984e-01,
         1.3629e-02, -1.5803e-02, -7.7371e-02,  6.9897e-02,  2.7974e-02,
         1.5132e-01, -9.9998e-01,  4.9626e-02, -4.2

In [129]:
mimic_dict['Calculated Total CO']

[-0.040763896,
 -0.074536584,
 -0.99783343,
 -0.9957536,
 0.96932846,
 0.99998224,
 0.03536505,
 -0.9999911,
 -0.062283672,
 0.0277831,
 0.1577978,
 -0.32055393,
 -0.9999999,
 0.9972942,
 -0.2622161,
 0.9458485,
 0.9951827,
 0.09787646,
 0.99873865,
 -0.10647794,
 -0.037276674,
 0.9990827,
 0.046975933,
 0.9999998,
 0.039822817,
 0.13674265,
 0.97183,
 -0.99905026,
 -0.092086926,
 -0.16956519,
 -0.18033169,
 -0.76073915,
 0.99995923,
 0.56337386,
 -0.15750876,
 0.16003972,
 0.09637019,
 -0.023353688,
 0.9922868,
 -0.9999974,
 0.06097747,
 -0.99502677,
 0.169131,
 0.13561203,
 0.9999994,
 -0.07477907,
 0.14935829,
 0.18821985,
 -0.014570755,
 -0.999922,
 -0.011881225,
 -0.9999999,
 -0.9999996,
 -0.9978742,
 0.93990856,
 0.032721795,
 0.9897085,
 -0.110521704,
 -0.9999987,
 0.109844975,
 0.0136293825,
 -0.015803315,
 -0.07737129,
 0.069897436,
 0.027974265,
 0.15132318,
 -0.9999752,
 0.04962597,
 -0.04272542,
 0.13620013,
 0.08783503,
 -0.104457594,
 -0.9966273,
 -0.12012492,
 -0.1749329